# Music Generation

<img src=https://media.giphy.com/media/xT8qBp0R5SxfLMIgjC/giphy.gif>





In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import editdistance

In [ ]:
# Remove index,title, source
with open('Data_Tunes.txt') as f:
    with open ('Cleaned_tunes.txt','w') as f_clean:
        for line in f:
            if line[:2] not in ["X:","T:",'% ','S:']:
                f_clean.write(line)

In [ ]:
# Read file
filename="Cleaned_tunes.txt"
with open(filename) as f:
  abc_file=f.read()

In [ ]:
# Define architecture
class LSTMNetwork(nn.Module):

  def __init__(self,unique_chars,n_layers=2,n_hidden=256,n_embed=100,drop_prob=0.5):
    super().__init__()
    self.drop_prob=drop_prob
    self.n_layers=n_layers
    self.n_hidden = n_hidden
    self.n_embed=n_embed

    self.embed=nn.Embedding(unique_chars,n_embed)
    self.lstm = nn.LSTM(n_embed, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
    self.dropout = nn.Dropout(drop_prob)
    self.fc = nn.Linear(n_hidden, unique_chars)
  
  def forward(self, x, hc):
    ''' Forward pass through the network. The inputs are x, and the hidden/cell state 'hc'. '''
    x = self.embed(x)
    x, (h, c) = self.lstm(x, hc)
    x = self.dropout(x)
    x = x.view(x.size()[0]*x.size()[1], self.n_hidden)
    x = self.fc(x)
    return x, (h, c)
        
  def init_hidden(self, n_seqs,cuda=True):
    ''' Initialize hidden state '''
    # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
    # initialized to zero, for hidden state and cell state of LSTM
    hidden=torch.zeros(self.n_layers, n_seqs, self.n_hidden)
    cell=torch.zeros(self.n_layers, n_seqs, self.n_hidden)

    if cuda:
      hidden,cell=hidden.cuda(),cell.cuda()
    return (hidden,cell)

In [ ]:
class MusicGenerator():
  
  def __init__(self,abc_file):
    self.file_data=abc_file
    self.char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(abc_file))))}
    self.index_to_char = {i: ch for (ch, i) in self.char_to_index.items()}
    self.data = np.asarray([self.char_to_index[c] for c in abc_file], dtype = np.int32)
    self.unique_chars=len(self.char_to_index)
    self.model=LSTMNetwork(unique_chars=self.unique_chars,n_hidden=256, n_layers=2,n_embed=100,drop_prob=0.25)

  
  def fit(self,epochs=50,n_seqs=16,n_steps=64,lr=0.001,device='GPU',cuda=True,val_frac=0.2,loss_criterion=nn.CrossEntropyLoss()):
    '''Train the LSTM network'''
    self.loss_criterion=loss_criterion
    train_data,val_data=self.train_val_data(val_frac)
    #Set the model in train mode    
    self.model.train()
    opt = torch.optim.Adam(self.model.parameters(), lr=lr)
    
    if cuda:
      self.model.cuda()

    for e in range(epochs):
      h = self.model.init_hidden(n_seqs,cuda)
      train_losses=[]
      for x,y in self.get_batches(train_data,n_seqs,n_steps):
        
        inputs,targets = torch.from_numpy(x).type(torch.cuda.LongTensor), torch.from_numpy(y).type(torch.cuda.LongTensor)

        if cuda:
          inputs,targets = inputs.cuda(),targets.cuda()

               
        h = tuple([each.data for each in h])
        #set all gradients to zero
        self.model.zero_grad()

        output,h=self.model.forward(inputs,h)

        # Calculate the forward losses
        loss = self.loss_criterion(output, targets.view(n_seqs*n_steps).type(torch.cuda.LongTensor))
        train_losses.append(loss.item())
        #backpropagate
        loss.backward()

        opt.step()

      val_losses=self.epoch_validation_loss(val_data,n_seqs,n_steps,cuda)
      self.print_epoch_loss(e,epochs,train_losses,val_losses)
        
  def train_val_data(self,val_frac):
    '''Split the data into a train and validation set'''
    val_idx = int(len(self.data)*(1-val_frac))
    train_data, val_data = self.data[:val_idx], self.data[val_idx:]
    return(train_data,val_data)
  
  def epoch_validation_loss(self,val_data,n_seqs,n_steps,cuda=False):
      '''Calculate validation loss at the end of each epoch'''
      val_h = self.model.init_hidden(n_seqs)
      val_losses = []
      
      for x, y in self.get_batches(val_data, n_seqs, n_steps):
          
          x, y = torch.from_numpy(x).type(torch.cuda.LongTensor), torch.from_numpy(y).type(torch.cuda.LongTensor)
          val_h = tuple([each.data for each in val_h])
          
          inputs, targets = x, y
          if cuda:
              inputs, targets = inputs.cuda(), targets.cuda()

          output, val_h = self.model.forward(inputs, val_h)
          val_loss = self.loss_criterion(output, targets.view(n_seqs*n_steps).type(torch.cuda.LongTensor))
      
          val_losses.append(val_loss.item())
      return val_losses

  def print_epoch_loss(self,e,epochs,train_losses,val_losses):
    '''Print average train and validation loss at the end of each epoch'''
    print("Epoch: {}/{}...".format(e+1, epochs),
                "Loss: {:.4f}...".format(np.mean(train_losses)),
                "Val Loss: {:.4f}".format(np.mean(val_losses)))

  
  def get_batches(self,data, n_seqs, n_steps):
    '''Generator to create batches of data'''
    batch_size = n_seqs * n_steps
    n_batches = len(data)//batch_size
    data = data[:n_batches * batch_size]
    data = data.reshape((n_seqs, -1))
    for n in range(0, data.shape[1], n_steps):
        x = data[:, n:n+n_steps]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], data[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], data[:, 0]
        yield x, y

  def predict_next_char(self, char, h=None, cuda=False, top_k=None,t=0.1):
    '''Predicts next character when generating a sequence'''
    if cuda:
        self.model.cuda()
    else:
        self.model.cpu()
    
    if h is None:
        h = self.model.init_hidden(1)
    
    x = np.array([[self.char_to_index[char]]])
    inputs = torch.from_numpy(x).type(torch.cuda.LongTensor)
    
    if cuda:
        inputs = inputs.cuda()
    
    h = tuple([each.data for each in h])
    out, h = self.model.forward(inputs, h)

    p = F.softmax(out/t, dim=1).data
    
    if cuda:
        p = p.cpu()
    
    if top_k is None:
        top_ch = np.arange(self.unique_chars)
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()
    
    p = p.numpy().squeeze()
        
    char = np.random.choice(top_ch, p=p/p.sum())
        
    return self.index_to_char[char], h
  
  def generate_music(self,max_len=200, prime='A', top_k=None, t=0.1,cuda=False,export_to_file=True,file_name='music.abc.txt'): 
    '''Generate a new music snippet'''
    if cuda:
        self.model.cuda()
    else:
        self.model.cpu()

    self.model.eval()
    
    chars = [ch for ch in prime]
    h = self.model.init_hidden(1)
    
    for ch in prime:
        char, h = self.predict_next_char(ch, h, cuda=cuda, top_k=top_k,t=t)

    chars.append(char)
    prev_char=chars[-1]
    
    for ii in range(max_len):
        char, h = self.predict_next_char(chars[-1], h, cuda=cuda, top_k=top_k)
        if (char=='\n')&(prev_char=='\n'):
          break
        prev_char=char
        chars.append(char)
    music = ''.join(chars)
    
    if export_to_file:
      with open(file_name, 'w') as file: 
        file.write(music)
      print(f"File written: {file_name}\n")
    
    print(f'Music generated:\n\n{music}')
    self.edit_distance(music)

  def edit_distance(self,music):
    '''Calculate Levenshtein distance for the new music generated'''
    songs=self.file_data.split('\n\n\n')
    song_distances,song_length = [],[]
    
    for song in songs:
      song_distances.append(editdistance.eval(song,music))
      song_length.append(len(song))
  
    print(f'\nMean length of input songs: {int(np.mean(song_length))}')
    print(f'Output song length: {len(music)}')
    print(f'Min Levenshtein distance: {np.min(song_distances)}')


In [ ]:
generator=MusicGenerator(abc_file)

In [ ]:
print("Model Architecture\n")
generator.model

Model Architecture



LSTMNetwork(
  (embed): Embedding(74, 100)
  (lstm): LSTM(100, 256, num_layers=2, batch_first=True, dropout=0.25)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=256, out_features=74, bias=True)
)

In [ ]:
generator.fit(epochs=30,n_seqs=128,n_steps=64,lr=0.001,cuda=True)

Epoch: 1/30... Loss: 3.6861... Val Loss: 3.3431
Epoch: 2/30... Loss: 3.0706... Val Loss: 3.2788
Epoch: 3/30... Loss: 2.8684... Val Loss: 3.0207
Epoch: 4/30... Loss: 2.5584... Val Loss: 2.8023
Epoch: 5/30... Loss: 2.2972... Val Loss: 2.6930
Epoch: 6/30... Loss: 2.0998... Val Loss: 2.6344
Epoch: 7/30... Loss: 1.9593... Val Loss: 2.6184
Epoch: 8/30... Loss: 1.8505... Val Loss: 2.5679
Epoch: 9/30... Loss: 1.7594... Val Loss: 2.5214
Epoch: 10/30... Loss: 1.6813... Val Loss: 2.4740
Epoch: 11/30... Loss: 1.6173... Val Loss: 2.4329
Epoch: 12/30... Loss: 1.5639... Val Loss: 2.4058
Epoch: 13/30... Loss: 1.5189... Val Loss: 2.3761
Epoch: 14/30... Loss: 1.4776... Val Loss: 2.3373
Epoch: 15/30... Loss: 1.4435... Val Loss: 2.3045
Epoch: 16/30... Loss: 1.4170... Val Loss: 2.2749
Epoch: 17/30... Loss: 1.3866... Val Loss: 2.2632
Epoch: 18/30... Loss: 1.3617... Val Loss: 2.2430
Epoch: 19/30... Loss: 1.3405... Val Loss: 2.2055
Epoch: 20/30... Loss: 1.3193... Val Loss: 2.2180
Epoch: 21/30... Loss: 1.3042.

In [ ]:
generator.generate_music(max_len=200, prime='M:6/8\nK:A\nP:A',t=0.5,top_k=5,cuda=True)

File written: music.abc.txt

Music generated:

M:6/8
K:A
P:A
|:A|"A"A2A A2A|"D"d2f fed|"A"c2c cBA|"D"d3 d2:|


Mean length of input songs: 306
Output song length: 62
Min Levenshtein distance: 102
